# Book Reading Propensity: Filtering the Data

In [1]:
import pandas as pd
import json
import os

## Filter the Data: 

In [47]:
data_file = "./data/goodreads_interactions_fantasy_paranormal.json"
# data_file_filtered = "./data/goodreads_interactions_fantasy_paranormal_2016.json"
# data_file_filtered_paired = "./data/goodreads_interactions_fantasy_paranormal_2016_pairs.json"
data_file_filtered_paired_csv = "./data/goodreads_interactions_fantasy_paranormal_2016_pairs.csv"

### Look at the first record

In [28]:
# Read a single record
with open(data_file, "r") as f:
    line = f.readline()
    print(line)
    record = json.loads(line)
    for k,v in record.items():
        v_upd = v if (v is not None and v != '') else 'NULL'
        print(f'{k}: {v_upd:>50}')

{"user_id": "8842281e1d1347389f2ab93d60773d4d", "book_id": "19161852", "review_id": "4443cb6883624c3772625ef5b7b4e138", "is_read": false, "rating": 0, "review_text_incomplete": "", "date_added": "Fri Sep 08 10:44:24 -0700 2017", "date_updated": "Fri Sep 08 10:44:24 -0700 2017", "read_at": "", "started_at": ""}

user_id:                   8842281e1d1347389f2ab93d60773d4d
book_id:                                           19161852
review_id:                   4443cb6883624c3772625ef5b7b4e138
is_read:                                                  0
rating:                                                  0
review_text_incomplete:                                               NULL
date_added:                     Fri Sep 08 10:44:24 -0700 2017
date_updated:                     Fri Sep 08 10:44:24 -0700 2017
read_at:                                               NULL
started_at:                                               NULL


### Question: Which field should we use as a timestamp?

In [39]:
%%time

cnt = 0
cnt_relevant = 0

mandatory_fields = ['date_updated', 'read_at', 'rating']

cnt_date_updated_null = 0
cnt_read_at_null = 0
rating_counts = {} 
default_rating = 0

with open(data_file, "r") as fr:

    for line in fr:
        # print(line)
        cnt += 1
        try:
            record = json.loads(line)
        except Exception as e:
            print(f'Failed to convert the line to a dict:\n{line}')
            print('Exception ----------------------------- \n', e)
            continue

        # Get only the mandatory columns
        record_mandatory_fields = {}
        for field in mandatory_fields:
            if field != 'rating':
                record_mandatory_fields[field] = record.get(field, "")
            else:
                record_mandatory_fields[field] = record.get(field, default_rating)

        # FILTER: Keep only the entries from the year 2016
        # combo = date_updated + ' ' + read_at
        combo_date = record_mandatory_fields['date_updated'] + ' ' + record_mandatory_fields['read_at']
        if '2016' in combo_date:
            cnt_relevant += 1
            if record_mandatory_fields['date_updated'] == '':
                cnt_date_updated_null += 1
            if record_mandatory_fields['read_at'] == '':
                cnt_read_at_null += 1
            
            rating_counts[record_mandatory_fields['rating']] = rating_counts.get(record_mandatory_fields['rating'], 0) + 1

            if cnt_relevant != 0 and cnt_relevant % 1000000 == 0:
                print(f'Relevant Record Count: {cnt_relevant:>10,d}. Total records inspected: {cnt:>10,d}')

print('-'*70)
print(f'Final Record Count (Input):           {cnt:>10,d}')
print(f'Final Relevant Record Count (Output): {cnt_relevant:10,d}')
print('-'*50)
print(f'NULL Count in "date_updated": {cnt_date_updated_null:10,d}')
print(f'NULL Count in "read_at": {cnt_read_at_null:10,d}')
print('-'*50)
for k,v in rating_counts.items():
    print(f'Rating {k:>}: {v:>10,d}')
print('-'*70)

Relevant Record Count:  1,000,000. Total records inspected:  7,191,867
Relevant Record Count:  2,000,000. Total records inspected: 13,981,481
Relevant Record Count:  3,000,000. Total records inspected: 20,156,727
Relevant Record Count:  4,000,000. Total records inspected: 25,717,217
Relevant Record Count:  5,000,000. Total records inspected: 30,759,259
Relevant Record Count:  6,000,000. Total records inspected: 35,226,872
Relevant Record Count:  7,000,000. Total records inspected: 38,337,797
Relevant Record Count:  8,000,000. Total records inspected: 42,209,869
Relevant Record Count:  9,000,000. Total records inspected: 50,634,669
----------------------------------------------------------------------
Final Record Count (Input):           55,397,550
Final Relevant Record Count (Output):  9,779,053
--------------------------------------------------
NULL Count in "date_updated":          0
NULL Count in "read_at":  7,214,855
Rating 0:  5,806,330
Rating 3:    764,680
Rating 5:  1,553,402
R

In [40]:
for k,v in sorted(rating_counts.items()):
    print(f'Rating {k:>}: {v:>10,d}')

Rating 0:  5,806,330
Rating 1:     69,845
Rating 2:    199,186
Rating 3:    764,680
Rating 4:  1,385,610
Rating 5:  1,553,402


### Answer: `date_updated`
 * In the time window of interest (year 2016) only the field `date_updated` appears to be a reliable timestamp (no missing values)
 * At the same time, the field `read_at` is riddled with NULLs

### Main part: Filter the records, and save them in a new file
 * Only the records from the year 2016
 * Only the records where the rating is an integer number greater than zero (`raiting = 0` means no rating was provided)

#### Option 1: JSON (Abandoned as inefficient and impractical)

#### Option 2: CSV

In [51]:
%%time

cnt = 0
cnt_relevant = 0

mandatory_fields = ['user_id', 'book_id', 'date_updated', 'rating']
default_rating = 0

with open(data_file, "r") as fr:
    with open(data_file_filtered_paired_csv, "w") as fw:
        # Write the header:
        fw.write(','.join(mandatory_fields) + '\n')
        # Read the input file, line by line
        for line in fr:
            cnt += 1
            try:
                record = json.loads(line)
            except Exception as e:
                print(f'Failed to convert the line to a dict:\n{line}')
                print('Exception ----------------------------- \n', e)
                continue
            
            # Get only the mandatory columns
            record_mandatory_fields = {}
            for field in mandatory_fields:
                if field != 'rating':
                    record_mandatory_fields[field] = record.get(field, "")
                else:
                    record_mandatory_fields[field] = record.get(field, default_rating)
            
            # FILTER: Keep only the entries from the year 2016
            combo_date = record_mandatory_fields['date_updated'] + ' ' + record.get('read_at', "")
            if ('2016' in record_mandatory_fields['date_updated']) and \
               (record_mandatory_fields['rating'] > 0):
                try:
                    # Add a line to the output file
                    record_csv_line = ','.join([str(record_mandatory_fields[field]) for field in mandatory_fields])
                    fw.write(record_csv_line + '\n')
                except Exception as e:
                    print(f'Failed to write the line to file:\n{line}')
                    print('Exception ----------------------------- \n', e)
                    continue
                cnt_relevant += 1
                    
                if cnt_relevant != 0 and cnt_relevant % 1000000 == 0:
                    print(f'Relevant Record Count: {cnt_relevant:>10,d}. Total records inspected: {cnt:>10,d}')

print('-'*70)
print(f'Final Record Count (Input):           {cnt:>10,d}')
print(f'Final Relevant Record Count (Output): {cnt_relevant:10,d}')
print('-'*70)

Relevant Record Count:  1,000,000. Total records inspected: 17,887,466
Relevant Record Count:  2,000,000. Total records inspected: 32,449,697
Relevant Record Count:  3,000,000. Total records inspected: 42,202,171
----------------------------------------------------------------------
Final Record Count (Input):           55,397,550
Final Relevant Record Count (Output):  3,813,915
----------------------------------------------------------------------
CPU times: user 3min 51s, sys: 4.32 s, total: 3min 56s
Wall time: 3min 58s


In [52]:
# TEST

count = 0
with open(data_file_filtered_paired_csv, "r") as f:
    for i, line in enumerate(f):
        count = i
        if i < 3:
            print(f'Record # {i}')
            print(line)
print(f'Final Record Count: {count+1:,}')

Record # 0
user_id,book_id,date_updated,rating

Record # 1
8842281e1d1347389f2ab93d60773d4d,29058155,Tue Dec 13 11:00:02 -0800 2016,3

Record # 2
8842281e1d1347389f2ab93d60773d4d,10673579,Fri Jan 08 01:49:56 -0800 2016,5

Final Record Count: 3,813,916


## Read the CSV, Add Time Features

In [78]:
df = pd.read_csv(data_file_filtered_paired_csv,
                dtype={'book_id':'Int64'})

In [79]:
df.shape

(3813915, 4)

In [80]:
df.head()

,user_id,book_id,date_updated,rating
0,8842281e1d1347389f2ab93d60773d4d,29058155,Tue Dec 13 11:00:02 -0800 2016,3
1,8842281e1d1347389f2ab93d60773d4d,10673579,Fri Jan 08 01:49:56 -0800 2016,5
2,8842281e1d1347389f2ab93d60773d4d,76620,Wed Dec 14 12:30:43 -0800 2016,5
3,7504b2aee1ecb5b2872d3da381c6c91e,29069989,Wed Aug 17 19:17:26 -0700 2016,2
4,7504b2aee1ecb5b2872d3da381c6c91e,8882815,Tue Jun 21 16:10:18 -0700 2016,3


In [81]:
df.dtypes

user_id         object
book_id          Int64
date_updated    object
rating           int64
dtype: object

In [82]:
datetime_format = '%a %b %d %H:%M:%S %z %Y'
df['ts'] = pd.to_datetime(df['date_updated'], format=datetime_format, errors='raise')
df.head()

,user_id,book_id,date_updated,rating,ts
0,8842281e1d1347389f2ab93d60773d4d,29058155,Tue Dec 13 11:00:02 -0800 2016,3,2016-12-13 11:00:02-08:00
1,8842281e1d1347389f2ab93d60773d4d,10673579,Fri Jan 08 01:49:56 -0800 2016,5,2016-01-08 01:49:56-08:00
2,8842281e1d1347389f2ab93d60773d4d,76620,Wed Dec 14 12:30:43 -0800 2016,5,2016-12-14 12:30:43-08:00
3,7504b2aee1ecb5b2872d3da381c6c91e,29069989,Wed Aug 17 19:17:26 -0700 2016,2,2016-08-17 19:17:26-07:00
4,7504b2aee1ecb5b2872d3da381c6c91e,8882815,Tue Jun 21 16:10:18 -0700 2016,3,2016-06-21 16:10:18-07:00


In [90]:
df['month'] = df['ts'].apply(lambda x: x.month)
df['day'] = df['ts'].apply(lambda x: x.day)
df['hour'] = df['ts'].apply(lambda x: x.hour)

df.head()

,user_id,book_id,date_updated,rating,ts,month,day,hour
0,8842281e1d1347389f2ab93d60773d4d,29058155,Tue Dec 13 11:00:02 -0800 2016,3,2016-12-13 11:00:02-08:00,12,13,11
1,8842281e1d1347389f2ab93d60773d4d,10673579,Fri Jan 08 01:49:56 -0800 2016,5,2016-01-08 01:49:56-08:00,1,8,1
2,8842281e1d1347389f2ab93d60773d4d,76620,Wed Dec 14 12:30:43 -0800 2016,5,2016-12-14 12:30:43-08:00,12,14,12
3,7504b2aee1ecb5b2872d3da381c6c91e,29069989,Wed Aug 17 19:17:26 -0700 2016,2,2016-08-17 19:17:26-07:00,8,17,19
4,7504b2aee1ecb5b2872d3da381c6c91e,8882815,Tue Jun 21 16:10:18 -0700 2016,3,2016-06-21 16:10:18-07:00,6,21,16


## Save the Final Dataset

In [92]:
df[['user_id','book_id','month','day', 'hour', 'rating']].to_csv('./data/goodreads_pairs_2016_enhanced.csv', 
                                                                 index=False)